In [2]:
import ImageReward as RM

model = RM.load("ImageReward-v1.0")

load checkpoint from /root/.cache/ImageReward/ImageReward.pt
checkpoint loaded


In [3]:
import os
import torch
import random
from pycocotools.coco import COCO

# 1. COCO annotations 파일 로드
def load_coco_annotations(annotation_file):
    coco = COCO(annotation_file)
    img_ids = coco.getImgIds()
    img_caption_mapping = {}

    for img_id in img_ids:
        img_info = coco.loadImgs(img_id)[0]

        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        img_file = img_info['file_name']
        captions = [ann['caption'] for ann in anns]
        img_caption_mapping[img_file] = captions
    
    return img_caption_mapping

def get_image_reward(image_dir, annotation_file, model_name):
    # COCO 데이터셋 로드
    img_caption_mapping = load_coco_annotations(annotation_file)
        
    # 각 이미지에 대해 랜덤 캡션을 사용하여 이미지 생성 및 저장
    num_image = 0
    reward_list = []
    for img_file, captions in img_caption_mapping.items():
        # 랜덤 캡션 선택
        prompt = random.choice(captions)
        prompt = captions[0]
        
        gen_image = os.path.join(model_name, f"generated_{num_image+1}.jpg")
        rewards = model.score(prompt, [gen_image])
        reward_list.append(rewards)
    
    print(f"Image Reward: {sum(reward_list)/len(reward_list)}")
        
# COCO annotations 파일 경로, 이미지 디렉토리, 출력 폴더 설정
annotation_file = "/workspace/LCM/data/annotations/captions_val2017.json"
image_dir = "/workspace/LCM/data/val2017"
model_name = "/workspace/LCM/generated_images/tcd-sdxl-lora_2"


# 이미지 생성 및 저장
get_image_reward(image_dir, annotation_file, model_name)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Image Reward: -2.1859815332718195
